In [6]:
import requests

url = "https://api-hoaxy.p.rapidapi.com/tweets"

querystring = {"ids":"%5B29317%2C 68363%5D"}

headers = {
    'x-rapidapi-host': "api-hoaxy.p.rapidapi.com",
    'x-rapidapi-key': "8bd7508073mshe5c51e555173ee1p1a803ajsnebd7566cb65b"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"error":"Empty of `ids`\nFormat error of `ids`","status":"ERROR"}



In [7]:
import http.client

conn = http.client.HTTPSConnection("api-hoaxy.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-hoaxy.p.rapidapi.com",
    'x-rapidapi-key': "8bd7508073mshe5c51e555173ee1p1a803ajsnebd7566cb65b"
    }

conn.request("GET", "/tweets?ids=%255B29317%252C%2068363%255D", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"error":"Empty of `ids`\nFormat error of `ids`","status":"ERROR"}



In [47]:
import http.client
import json

conn = http.client.HTTPSConnection("api-hoaxy.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-hoaxy.p.rapidapi.com",
    'x-rapidapi-key': "8bd7508073mshe5c51e555173ee1p1a803ajsnebd7566cb65b"
    }

conn.request("GET", "/articles?sort_by=relevant&use_lucene_syntax=true&query=Bernie+Sanders+Is+A+Nazi", headers=headers)

res = conn.getresponse()
data = res.read()
JSON = json.loads(data.decode("utf-8"))

for item in JSON["articles"]:
    print(item["title"])
    print(item["site_type"])
    print(item["score"])

These 13 Corrupt Democrats Stopped Bernie Sanders’ Amendment
claim
190.8426513672
Bernie Sanders, FDR, and Contested Conventions
fact_checking
205.6647033691
Could Bernie Sanders Win the Presidency as a Write-In Candidate?
fact_checking
191.4320068359
Bernie Sanders campaign fined for 2016 foreign collusion
claim
192.289276123
Bernie Sanders: Trump is Tough on Children But Not On Putin
claim
191.4777832031
DNC Ban Bernie Sanders From Running In 2020
claim
196.0847167969
Bernie Sanders: 'Abolish the Electoral College'
claim
196.9642028809
WikiLeaks: Bernie Sanders Was A Plant, Democratic Primaries Rigged
claim
192.4945526123
Bernie Sanders Announces He Will Seek Reelection
claim
193.772857666
Bernie Sanders Bombshell: ‘Saudi Arabia Funds ISIS’
claim
193.8087158203
Bernie Sanders’ ‘heartless spouse’ a greedy landlord?
claim
202.3274230957
The Bernie Sanders Campaign is Imploding
claim
213.0172119141
Bernie Sanders Backtracks After Blaming Trump for Charlottesville
claim
191.9380645752
Be